<a href="https://colab.research.google.com/github/Neurocoders/AttentionProject/blob/branch2/load_steinmetz_data_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading of Steinmetz data

includes some visualizations

In [ ]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

fname.append('steinmetz_st.npz')
fname.append('steinmetz_wav.npz')
fname.append('steinmetz_lfp.npz')


url.append("https://osf.io/4bjns/download")
url.append("https://osf.io/ugm9v/download")
url.append("https://osf.io/kx3v9/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [ ]:
import numpy as np

alldat = np.array([])
for j in range(3):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]

dat_LFP = np.load('steinmetz_lfp.npz', allow_pickle=True)['dat']
dat_LFP = dat_LFP[11] #beware of lfp vs LFP here
print(dat_LFP.keys())
dat_WAV = np.load('steinmetz_wav.npz', allow_pickle=True)['dat']
dat_WAV = dat_WAV[11]
print(dat_WAV.keys())

print(dat.keys())

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 


dict_keys(['lfp', 'lfp_passive', 'brain_area_lfp'])
dict_keys(['waveform_w', 'waveform_u', 'trough_to_peak'])
dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


`alldat` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `dat = alldat[k]`, you have the following fields:

* `dat['mouse_name']`: mouse name
* `dat['date_exp']`: when a session was performed
* `dat['spks']`: neurons by trials by time bins.    
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `dat['contrast_left']`: contrast level for left stimulus. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_times']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `dat['wheel']`: exact position of the wheel that the mice uses to make a response, binned at `10ms`. 
* `dat['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position. 
* `dat['lfp']`: recording of the local field potential in each brain area from this experiment, binned at `10ms`.
* `dat['brain_area_lfp']`: brain area names for the LFP channels. 
* `dat['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons". 
* `dat['waveform_w']`: temporal components of spike waveforms. `w@u` reconstructs the time by channels action potential shape. 
* `dat['waveform_u]`: spatial components of spike waveforms.
* `dat['%X%_passive']`: same as above for `X` = {`spks`, `lfp`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses. 




In [ ]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [ ]:
#@title Run this cell if you want to see the shape of different data dictionary fields

for keys in dat_LFP:
    try:
        print(f"the shape of {keys} is:", dat_LFP[keys].shape)
    except:
        print(f"{keys}:", dat_LFP[keys])
        
for keys in dat: # see the shape of contents of "dat" variable
    try:
        print(f"the shape of {keys} is:", dat[keys].shape)
    except:
        print(f"{keys}:", dat[keys])
        

the shape of lfp is: (12, 340, 250)
the shape of lfp_passive is: (12, 110, 250)
brain_area_lfp: ['DG', 'LGd', 'SUB', 'VISp', 'ACA', 'MOs', 'PL', 'CA1', 'DG', 'LH', 'MD', 'VISam']
the shape of spks is: (698, 340, 250)
the shape of wheel is: (1, 340, 250)
the shape of pupil is: (3, 340, 250)
the shape of response is: (340,)
the shape of response_time is: (340, 1)
bin_size: 0.01
stim_onset: 0.5
the shape of contrast_right is: (340,)
the shape of contrast_left is: (340,)
the shape of brain_area is: (698,)
the shape of feedback_time is: (340, 1)
the shape of feedback_type is: (340,)
the shape of gocue is: (340, 1)
mouse_name: Lederberg
date_exp: 2017-12-05
the shape of trough_to_peak is: (698,)
the shape of active_trials is: (450,)
the shape of contrast_left_passive is: (110,)
the shape of contrast_right_passive is: (110,)
the shape of spks_passive is: (698, 110, 250)
the shape of pupil_passive is: (3, 110, 250)
the shape of wheel_passive is: (1, 110, 250)
the shape of prev_reward is: (340,

In [ ]:
#@title Import required libraries
# please use the same conventions across the document
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.signal as signal
import scipy.fftpack as fftpack  

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#@title Data splitting
def split_stim_indices(dat):
  n_trials = len(dat['response'])
  no_stim = np.logical_and(dat['contrast_left'] == 0, dat['contrast_right'] == 0)
  one_side_stim = np.logical_xor(dat['contrast_left'] == 0, dat['contrast_right'] == 0)
  two_side_stim = np.logical_and(dat['contrast_left'] > 0, dat['contrast_right'] > 0)
  return (no_stim, one_side_stim, two_side_stim)

def split_accuracy_indices(dat):
  n_trials = len(dat['response'])
  correct = []
  incorrect = []
  for i in range(n_trials):
    if (dat['contrast_left'][i] > dat['contrast_right'][i] and dat['response'][i] == 1 ):
      correct.append( True )
      incorrect.append(False)
    elif (dat['contrast_left'][i] < dat['contrast_right'][i] and dat['response'][i] == -1 ):
      correct.append( True )
      incorrect.append(False)
    elif (dat['contrast_left'][i] == 0 and dat['contrast_right'][i] == 0 and dat['response'][i] == 0 ):
      correct.append( True )
      incorrect.append(False)
    else:
      incorrect.append( True )
      correct.append(False)
  return (correct, incorrect)

no_stim, one_side_stim, two_side_stim = split_stim_indices(dat)
correct, incorrect = split_accuracy_indices(dat)
#to get only one_side trials which are correct use np.logical_and(one_side_stim, correct)
correct_one_side = np.logical_and(one_side_stim, correct)


In [ ]:
eff_response = dat["response_time"]-dat["gocue"]
plotter = dat["response_time"]
sns.distplot(plotter, hist=True, kde=True,  # plot histogram with kernel density
             bins=80, color = 'gray', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4, 'color': 'k'});
sns.distplot(plotter[correct], hist=True, kde=True,  # plot histogram with kernel density
             bins=80, color = 'gray', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4, 'color': 'r'});
sns.distplot(plotter[incorrect], hist=True, kde=True,  # plot histogram with kernel density
             bins=80, color = 'gray', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4, 'color': 'g'});
sns.distplot(plotter[two_side_stim], hist=True, kde=True,  # plot histogram with kernel density
             bins=80, color = 'gray', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4, 'color': 'b'});

**Considerations:**

1- go cue time, response time and feedback time are measured with respect to the same time reference point (have the same 0 time)

2- cannot understand the "reaction time" variable

3- have some infinity values in the reaction time! (confirm this?)

In [ ]:
#------------ Initial required variables, gather them in here if possible--------------

dt = 10 # binning at 10 ms
NT = dat_LFP['lfp'].shape[-1] # number of time bins
time_vec = dt * np.arange(NT)
total_trial = dat_LFP['lfp'].shape[1] # number of trials

# lfp has form brain area* trial * time

In [ ]:
def visualize_areas(data_LFP = dat_LFP,trial_num = 1,typ = "hamming", win_len = 6, 
                 data_all = dat, fig_size = (18,10)):
    """
    Returns two indices corresponding to the user defined brain regions to study LFP
  
    Args:
    data_LFP: dictionary of LFPs
    trial_num : Trial number to visualize
    typ:  type of filter window, choose from gaussian, hamming, and hann
    win_len: length of the filter window
    data_all: Contains the reaction times, go cue and other trail information 

    output:
    ind1: index of the first brain region that we want to study
    ind2: index of the second brain region that we want to study
    area1: name of the first brain region that we want to study
    area2: name of the second brain region that we want to study

    """
    print(data_LFP["brain_area_lfp"])
    area1 = input("please insert the first lfp region to plot:")
    area2 = input("please insert the second lfp region to plot:")
    
    ind1 = data_LFP["brain_area_lfp"].index(area1)
    ind2 = data_LFP["brain_area_lfp"].index(area2)
    regions = np.array(dat_LFP["brain_area_lfp"]) # this is for multiple occurences of one region
    
    # see if there are multiple recordings of one area
    if data_LFP["brain_area_lfp"].count(area1) >1:
        what_area1 = int(input("more than region with this name was found,\
        please specify which instance you want to see (1: first, 2:second,...)"))
        ind1 = np.where(regions == area1)[0][what_area1-1]

    if data_LFP["brain_area_lfp"].count(area2) >1:
        what_area2 = int(input("more than region with this name was found,\
        please specify which instance you want to see (1: first, 2:second,...)"))
        ind2 = np.where(regions == area2)[0][what_area2-1]

    # visualization of raw LFPs before filtering
    temp_inp = input("do you wish to visualize the 2 LFPs that you want to analyse? y/n")
    if temp_inp == "y":
        plt.figure(figsize = (10,7))
        plt.plot(time_vec, data_LFP["lfp"][ind1, trial_num,:], label = area1 );
        plt.plot(time_vec, data_LFP["lfp"][ind2, trial_num, :], label = area2)

        plt.axvline(x = (data_all["gocue"])[trial_num] * 1000, color = 'black', label = "go cue") # need the go cue and response/reaction time 
        plt.axvline(x = (data_all["response_time"])[trial_num] * 1000,             # from the main data
                ls = '--', color = 'black', label = "response time")

        plt.xlabel("time(ms)", fontsize = 12)
        plt.ylabel("voltage", fontsize = 12)
        plt.title("plot of two unfiltered LFPs", fontsize = 13)
        plt.legend()
        plt.show()

        # smoothing filtering
        # choosing the window type
    if typ == "hann":
        win = signal.windows.hann(win_len, sym=True)
    elif typ == "gaussian":
        st = float(input("you have chosen gaussian, please specify a number for gaussian filter's standard deviation"))
        win = signal.windows.gaussian(win_len, st, sym=True)
    
    elif typ == "hamming":
        win = signal.windows.hamming(win_len, sym=True)

    else:
        raise ValueError("type is not  hann, hamming, gaussian")

     # applying the filter using filtfilt    
    smooth_sign1 = signal.filtfilt(b = win, a = 1, x = data_LFP["lfp"][ind1, trial_num, :], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)
    smooth_sign2 = signal.filtfilt(b = win, a = 1, x = data_LFP["lfp"][ind2, trial_num, :], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)

    # plotting filter results
    temp_inp2 = input("do you wish to visualize the results of your smoothing? y/n")
    if temp_inp2 == 'y':
        fig, axs = plt.subplots(1,2,figsize = fig_size)
        
        axs[0].plot(time_vec, data_LFP["lfp"][ind1, trial_num, :],label = 'Raw Signal')
        axs[0].plot(time_vec, smooth_sign1,label = 'Smoothened signal')
        axs[0].set(title = f"LFP from {area1}")
        
        #plt.figure(figsize = (10,7))
        axs[1].plot(time_vec,data_LFP["lfp"][ind2, trial_num, :],label = 'Raw Signal')
        axs[1].plot(time_vec,smooth_sign2,label = 'Smoothened signal')
        axs[1].set(title = f"LFP from {area2}")
        plt.xlabel('Time'); plt.ylabel('Amplitude')
        plt.legend(); plt.show()

    return smooth_sign1, smooth_sign2, area1, area2, ind1, ind2

    
#@title Get Filtered LFP data for the interested brain areas
smooth_sig1, smooth_sig2, area1, area2, ind1, ind2 = visualize_areas(data_LFP = dat_LFP,trial_num = 1,typ = "hann", win_len = 4, 
                                                         data_all = dat, fig_size = (18,10))
        

['DG', 'LGd', 'SUB', 'VISp', 'ACA', 'MOs', 'PL', 'CA1', 'DG', 'LH', 'MD', 'VISam']
please insert the first lfp region to plot:VISam
please insert the second lfp region to plot:LGd
do you wish to visualize the 2 LFPs that you want to analyse? y/nn
do you wish to visualize the results of your smoothing? y/nn


In [ ]:
#@ ---------------Fourier Transform--------------------
# NT => number of time points, NT/2 would be the nyquest 
# dt => time bin size
dt = 10
# time_vec => time vector in ms
NT = dat_LFP['lfp'].shape[-1] # number of time bins
time_vec = dt * np.arange(NT)

def fourier(NT = NT, dt = dt, signal1 = smooth_sig1, signal2 =  smooth_sig2,
            dat = dat_LFP,trial_num = 1, area1 = area1,
                area2 = area2, fig_size = (18,10)):
    """
   

    This function performs fourier transformation using FFT on the lfp signal and also
    plots the time series and frequency result of fourier. dat[region, trial_num, :]
    
    Inputs:
    NT: number of total time points
    dt: size of time bin
    signal1: first time seri to decompose, set to the first output of the smoothing by default
    signal2: second time seri to decompose, set to the second output of the smoothing by default
    dat: dictionary containing the lfp data
    trial_num: number of trial to plut, 
    

    Returns:
    hz, amp1, amp2
    hz: frequency vector (hz = np.linspace(0,sampl_rate, NT) )
    amp1: amplitude result of fourier transform of area 1 LFP
    amp2: amplitude result of fourier transform of area 1 LFP
    * both only contain elements up to the nyquist frequency 
    """

    sampl_rate = 1/(dt/1000) # sampling rate
    nyquist = sampl_rate/2


    freq_seri1 = fftpack.fft(signal1) #compute freq seri
    freq_seri2 = fftpack.fft(signal2)

    hz = np.linspace(0,sampl_rate, NT) # frequency vector same for all
   
    amp1 = 2.0/NT * np.abs(freq_seri1)  # amplitude vector,
    # 2 * is to add the amplitude from  negative frequencies to positive ones
    # divide by number of time points for normalization
    amp2 = 2.0/NT * np.abs(freq_seri2)
    
    
    # plot timeseries
    # temp_inp = input("do you want to visualize the results of your decomposition? y/n")

    # if temp_inp == 'y':
    #     fig, axs = plt.subplots(1,2, figsize = fig_size)

    #     axs[0].plot(hz, amp1) # this will have both neg and pos freqs so we have to limit it to nyquist
    #     axs[0].set(ylabel='Amplitude',xlabel='Frequency (Hz)',title= f'FFT result from {area1}', xlim = [0, nyquist])  
    #     axs[1].plot(hz, amp2) # this will have both neg and pos freqs so we have to limit it to nyquist
    #     axs[1].set(ylabel='Amplitude',xlabel='Frequency (Hz)',title= f'FFT result from {area2}', xlim = [0, nyquist])
        
    #     plt.show()

    # now we want to only return the elements of hz and amp that 
    # up to the nyquist: this is to find the index of nearest element of hz to nyquist   
    return_ind = int(np.where(abs(hz-nyquist) == np.min(abs(hz - nyquist)))[0][0])+1
    # pdb.set_trace()

    return hz[:return_ind], amp1[:return_ind], amp2[:return_ind]

freq_vals,fft_signal1,fft_signal2 = fourier()

In [ ]:
b = np.array([])

In [ ]:
# print(freq_vals)
# len(dat['response'])
correct_trials    = np.where(correct_one_side == True)[0]
incorrect_trials  = np.where(correct_one_side == False)[0]
alpha_indices = np.where(np.logical_and(freq_vals>=8, freq_vals<=12)) # alpha - 8-12 Hz # get indices where frequency values are in alpha and theta range
theta_indices = np.where(np.logical_and(freq_vals>=4, freq_vals<=7))
a = np.mean(fft_signal1[alpha_indices])

In [ ]:
# Obtain powers of alpha and theta band

# Correct trials for stimulus on one side
incorrect_one_side = np.logical_and(one_side_stim,incorrect)
correct_trials    = np.where(correct_one_side == True)[0]
incorrect_trials  = np.where(incorrect_one_side == True)[0]


def get_powers(area1='VISam',area2='LGd', dat_LFP= dat_LFP,dat = dat,correct_trials = correct_trials, incorrect_trials = incorrect_trials):
  """
  This function takes the areas into account and computes the band powers by obtaining the 
  power values at every time bin (in a particular frequency band) and then takes the 
  average of those power values for each trial

  input arguments:
  area1 : Brain area 1
  area2 : Brain area 2
  dat_LFP: LFP data
  dat: dictionary containing response times, time vectors
  correct_trials: correct trials for one sided stimulus
  incorrect_trials: incorrect trials for one sided stimulus

  output arguments:
  correct_trials_alpha_power_signal1/2 : Alpha power for correct trials for signal 1/ 2
  incorrect_trials_alpha_power_signal1/2 : Alpha power for incorrect trials for signal 1/2
  correct_trials_theta_power_signal1/2: Theta power for correct trials for signal 1/2
  incorrect_trials_theta_power_signal1/2: Theta power for incorrect trials for signal 1/2 

  """
  # Initializing empty arrays to append
  correct_trials_alpha_power_signal1 = np.array([]) ; correct_trials_alpha_power_signal2 = np.array([]);
  correct_trials_theta_power_signal1 = np.array([]) ; correct_trials_theta_power_signal2 = np.array([]);
  incorrect_trials_alpha_power_signal1 = np.array([]); incorrect_trials_alpha_power_signal2 = np.array([])
  incorrect_trials_theta_power_signal1 = np.array([]); incorrect_trials_theta_power_signal2 = np.array([])

  ind1 = dat_LFP["brain_area_lfp"].index(area1)
  ind2 = dat_LFP["brain_area_lfp"].index(area2)
  regions = np.array(dat_LFP["brain_area_lfp"]) # this is for multiple occurences of one region

  # If more than one instance, taking the first recording area into account
  if dat_LFP["brain_area_lfp"].count(area1) >1:
       ind1 = np.where(regions == area1)[0][1]

  if dat_LFP["brain_area_lfp"].count(area2) >1:
      ind2 = np.where(regions == area2)[0][1]
  
  for i_trial in range(len(dat['response'])):
    # Compute Fourier transform 
    freq_vals, fft_signal1, fft_signal2 = fourier(NT = NT, dt = 10, signal1 = dat_LFP["lfp"][ind1,i_trial,:] , signal2 = dat_LFP["lfp"][ind2,i_trial,:],
                                                  dat = dat_LFP,trial_num = i_trial, area1 = area1, area2 = area2, fig_size = (18,10))
    # fft_signal1 = 10 * np.log10(fft_signal1) # In dB, can use to visualize power spectrum but powers are calculated using raw power 
    # fft_signal2 = 10 * np.log10(fft_signal2)
    alpha_indices = np.where(np.logical_and(freq_vals>=8, freq_vals<=12)) # alpha - 8-12 Hz # get indices where frequency values are in alpha and theta range
    theta_indices = np.where(np.logical_and(freq_vals>=4, freq_vals<=7)) # theta - 4-7 Hz

    if i_trial in correct_trials:
      correct_trials_alpha_power_signal1 =  np.append(correct_trials_alpha_power_signal1, np.mean(fft_signal1[alpha_indices]))
      correct_trials_alpha_power_signal2 = np.append(correct_trials_alpha_power_signal2, np.mean(fft_signal2[alpha_indices]))

      correct_trials_theta_power_signal1 = np.append(correct_trials_theta_power_signal1, np.mean(fft_signal1[theta_indices]))
      correct_trials_theta_power_signal2 = np.append(correct_trials_theta_power_signal2, np.mean(fft_signal2[theta_indices]))
   
    elif i_trial in incorrect_trials:
      incorrect_trials_alpha_power_signal1 = np.append(incorrect_trials_alpha_power_signal1, np.mean(fft_signal1[alpha_indices])) 
      incorrect_trials_alpha_power_signal2 = np.append(incorrect_trials_alpha_power_signal2, np.mean(fft_signal2[alpha_indices]))

      incorrect_trials_theta_power_signal1 = np.append(incorrect_trials_theta_power_signal1, np.mean(fft_signal1[theta_indices]))
      incorrect_trials_theta_power_signal2 = np.append(incorrect_trials_theta_power_signal2, np.mean(fft_signal2[theta_indices]))
  
  return correct_trials_alpha_power_signal1, correct_trials_alpha_power_signal2,correct_trials_theta_power_signal1,correct_trials_theta_power_signal2,\
  incorrect_trials_alpha_power_signal1,incorrect_trials_alpha_power_signal2, incorrect_trials_theta_power_signal1, incorrect_trials_theta_power_signal2


# Function ends


# correct_trials_alpha_power_signal1, correct_trials_alpha_power_signal2,correct_trials_theta_power_signal1,correct_trials_theta_power_signal2,incorrect_trials_alpha_power_signal1,incorrect_trials_alpha_power_signal2, incorrect_trials_theta_power_signal1, incorrect_trials_theta_power_sign = get_powers()
correct_trials_alpha_power_signal1, correct_trials_alpha_power_signal2,correct_trials_theta_power_signal1,correct_trials_theta_power_signal2,\
incorrect_trials_alpha_power_signal1,incorrect_trials_alpha_power_signal2, incorrect_trials_theta_power_signal1, \
incorrect_trials_theta_power_signal2 = get_powers(area1='VISam',area2='LGd', dat_LFP= dat_LFP,dat = dat,correct_trials = correct_trials,
                                                  incorrect_trials = incorrect_trials)


In [ ]:
# plt.scatter(correct_trials_alpha_power_signal1,correct_trials_theta_power_signal1)
# plt.show()
# from scipy import stats
# corr, pVal = stats.pearsonr(incorrect_trials_alpha_power_signal1,incorrect_trials_theta_power_signal1)
# print(corr,pVal)
# print(dat["response"][incorrect_trials], dat["response"][correct_trials])
par1 = correct_trials_theta_power_signal1 - correct_trials_alpha_power_signal1 ** -1
par2 = correct_trials_theta_power_signal1 ** 2

corr,pVal = stats.pointbiserialr(par, dat["response"][correct_trials])
print(corr,pVal)
# So the square of theta powers for VISam are correlated with the correct response?

# Creating a dictionary to store parameters
trial_params = {
    "responses": dat["response"][correct_trials]
    "correct_trials_theta_power_signal1": correct_trials_theta_power_signal1
    "correct_trials_alpha_power_signal1": correct_trials_alpha_power_signal1
}

0.22618671768425094 0.018580557499265016


In [ ]:
# Bayesian regressor - to try with regularizer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
import numpy as np
X = np.vstack([correct_trials_alpha_power_signal1,correct_trials_theta_power_signal1])
y = dat["response"][correct_trials]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
reg = linear_model.BayesianRidge()
reg.fit(X.T,y)#(X.reshape(-1, 1), y)
a =X.T
print(reg.predict(a[8:10,:])) # Testing with random samples
print( y[8:10]) # actual response

In [ ]:
# THE DECODER.......................................................
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

sampl_rate = 1/(dt/1000) # sampling rate
nyquist = sampl_rate/2
hz = np.linspace(0,sampl_rate, NT) # frequency vector same for all

alpha_indices = np.where(np.logical_and(hz>=8, hz<=12)) # alpha - 8-12 Hz # get indices where frequency values are in alpha and theta range
theta_indices = np.where(np.logical_and(hz>=4, hz<=7)) 

incorrect_one_side = np.logical_and(one_side_stim,incorrect)
correct_trials    = np.where(correct_one_side == True)[0]
incorrect_trials  = np.where(incorrect_one_side == True)[0]

for i_area in range(12):
  alpha_power = np.array([]); theta_power = np.array([])
  
  for i_trial in range(len(dat['response'])):
    freq_seri1 = fftpack.fft(dat_LFP['lfp'][i_area][i_trial]) #compute freq series
    amp1 = 2.0/NT * np.abs(freq_seri1)  # amplitude vector,
    return_ind = int(np.where(abs(hz-nyquist) == np.min(abs(hz - nyquist)))[0][0])+1
    amp1 = amp1[:return_ind]
    if i_trial in correct_trials:
      alpha_power = np.append(alpha_power, np.mean(amp1[alpha_indices]))
      theta_power = np.append(theta_power, np.mean(amp1[theta_indices]))

  X = np.vstack([alpha_power,theta_power])  # Naive Bayesian decoder
  y = dat["response"][correct_trials]
  X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.5)
  gnb = GaussianNB()
  y_pred = gnb.fit(X_train, y_train).predict(X_test)
  print(dat_LFP['brain_area_lfp'][i_area],':',accuracy_score(y_test,y_pred)* 100)
